# Calculating statistics on Sentinel 1 GRD data

The aim of this mini project is to take a monthly aggregate of Sentinel 1 data for a location and compute the following:

- Q10
- Q50
- Q90
- STDDEV
- Range of values

The purpose of this script is to use it as demonstrator for the OGC Application package which can be run on EOEPCA

## Current status
Currently the script extracts 1 image from SentinelHub and computes the statistics. In the future the monthly aggregate needs to be added


In [ ]:
from sentinelhub import SHConfig, SentinelHubCatalog, SentinelHubDownloadClient, SentinelHubRequest, BBox, CRS, DataCollection, MimeType
from sentinelhub import CRS, BBox, DataCollection, SHConfig

import requests

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import rasterio as rio
from rasterio.plot import show
from rasterio.mask import mask

In [ ]:
# Client secrets for SentinelHub
# Ideally should be safely stored in env file but where is the fun in that
# FIX BEFORE PUBLISHING TO GITHUB
client_id = '6ce93625-2a00-4f57-b761-3012307d7cb3'
client_secret = '96dd90d8-6547-4187-9'
instance_id = '55b1d947-7b59-4bcd-a42e-fdbf8b9691e6'

# Initiate the configuration file of SentinelHub and add all of the variables
config = SHConfig()
config.instance_id = instance_id
config.sh_client_id = client_id
config.sh_client_secret = client_secret



In [ ]:
# Initialize the catalog variable
catalog = SentinelHubCatalog(config=config)

In [ ]:
# Print some information about the Catalog (version, endpoints etc.)
catalog.get_info()

In [ ]:
# Get all of the collections from the Catalog
collections = catalog.get_collections()

In [ ]:
# Sanity check for now, remove for production
print(collections)

In [ ]:
# Print information about the Sentinel 1 GRD dataset
DataCollection.SENTINEL1

In [ ]:
# Define a bounding box and time interval
# Search the catalog for the data
# Show the results
BBOX = BBox((-87.72171, 17.11848, -87.342682, 17.481674), crs=CRS.WGS84)
time_interval = "2019-01-01", "2019-02-01"

search_iterator = catalog.search(
    DataCollection.SENTINEL1,
    bbox=BBOX,
    time=time_interval,
    #filter="",
    fields={"include": ["id", "properties.datetime"], "exclude": []},
)

results = list(search_iterator)
print("Total number of results:", len(results))

results

In [ ]:
evalscript = '''
//VERSION=3

function setup() {
  return {
    input: [
      {
        bands: ["VV","VH"],                  
      }
    ],
    output: [
      {
        id: "default",
        bands: 1,
        sampleType: "AUTO",        
      },    
    ],
    mosaicking: "SIMPLE",
  };
}


function evaluatePixel(sample) {
        return [sample.VV];
}
  
'''
requestdata = SentinelHubRequest(
    data_folder='testing',
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
        DataCollection.SENTINEL1,
        time_interval=('2019-01-01','2019-02-01')
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=BBOX,
    size=[512, 343.697],
    config=config,
)

In [ ]:
%%time
sentinel_data = requestdata.get_data(save_data=True)

In [ ]:
print(f"Returned data is of type = {type(sentinel_data)} and length {len(sentinel_data)}.")
print(f"Single element in the list is of type {type(sentinel_data[-1])} and has shape {sentinel_data[-1].shape}")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

image = sentinel_data[0]
print(f"Image type: {image.dtype}")
# Show the image to see if its working
plt.imshow(image)

In [72]:
# Calculate statistics 
# Yielding percentiles (Q10, Q50, Q90), STDDEV, range of values
stddev = np.nanstd(sentinel_data)
mean = np.nanmean(sentinel_data)
quant10 = np.quantile(sentinel_data, 0.1)
quant50 = np.quantile(sentinel_data, 0.5)
quant90 = np.quantile(sentinel_data, 0.9)
r_o_values = np.ptp(sentinel_data)

